In [23]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` node.states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [24]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

# Search Algorithms: Astar and Breadth First Search

In [25]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def g(n): return n.path_cost

def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


# Octa Puzzle Problems

The next code block contains solution to the following problems:

1. Your first task is to define the Octa puzzle as a state space problem. Design a staterepresentation to model the Octa puzzle. Your state representation should model both the‘octagon’ shaped grid shown above and the first sixteen numbers that are arranged on theoctagon shaped grid. Make sure your state representation is easy to visualize and understand(see question 4 below). Explain your state representation in a Markdown cell in your JupyterNotebook following the explanation style (using simple pictures and examples) used in AIMA-Python Search Jupyter Notebook.

2. Define the OctaPuzzle class extending the abstract Problem class defined in AIMA-Python. YourOctaPuzzle class should use the state representation you design in the above question. You arerequired to provide implementations for actions and result methods. The abstract Problem classdefines a simple implementation of the is_goal method where a state is directly matchedagainst the goal state. Depending upon your state representation of the OctaPuzzle you mayneed to provide an implementation of the is_goal method as well to override the existingimplementation. Add all your explanations of code directly into your Jupyter Notebook in amarkdown cell. Your OctaPuzzle class should be generic and should allow you to use it ondifferent problem instances that involve different levels of complexity as explained further inthe questions below.



In [33]:
import random
from turtle import right

class OctaPuzzle(Problem):
    def __init__(self, initial):
        self.initial = initial
        self.goal = {}
    
    def actions(self, state):
        moves = []
        for i in range(16):
            if i not in self.initial:
                moves.append(i)
        return tuple(moves)

    def result(self, state, action):
        s = list(state)
        try:
            blank = s.index(0)
            if not action in s:
                s[blank] = action
        except ValueError:
            make_blank = s.index(action)
            s[make_blank] = 0
        return tuple(s)

    def is_goal(self, state):
        # Goal: (9,3,13,6,12,11,5,1,14,16,2,4,7,8,15,10)
        top_horizontal, bottom_horizontal, left_vertical, right_vertical, top_left, top_right, left_bottom, right_bottom, diamond_corners, square_corners = goal_state(state)
        
        return top_horizontal == 34 and bottom_horizontal == 34 and left_vertical == 34 and right_vertical == 34 and top_left == 34 and top_right == 34 and left_bottom == 34 and right_bottom == 34 and diamond_corners == 34 and square_corners == 34
    
    def h1(self, node):
        cost = 10
        top_horizontal, bottom_horizontal, left_vertical, right_vertical, top_left, top_right, left_bottom, right_bottom, diamond_corners, square_corners = goal_state(node.state)
        
        cost = cost_per_node(top_horizontal, cost) + cost_per_node(bottom_horizontal, cost) + cost_per_node(left_vertical, cost) + cost_per_node(right_vertical, cost) + cost_per_node(top_left, cost) + cost_per_node(top_right, cost) + cost_per_node(left_bottom, cost) + cost_per_node(right_bottom, cost) + cost_per_node(diamond_corners, cost) + cost_per_node(square_corners, cost)

        return cost
    
    def h(self, node): return self.h1(node)

def goal_state(state):
    top_horizontal = sum([state[1], state[2], state[3], state[4]])
    bottom_horizontal = sum([state[11], state[12], state[13], state[14]])
    left_vertical = sum([state[1], state[5], state[9], state[11]])
    right_vertical = sum([state[4], state[6], state[10], state[14]])
    top_left = sum([state[0], state[2], state[5], state[7]])
    top_right = sum([state[0], state[3], state[6], state[8]])
    left_bottom = sum([state[7], state[9], state[12], state[15]])
    right_bottom = sum([state[8], state[10], state[13], state[15]])
    diamond_corners = sum([state[0], state[7], state[8], state[15]])
    square_corners = sum([state[1], state[4], state[11], state[14]])
    
    return top_horizontal, bottom_horizontal, left_vertical, right_vertical, top_left, top_right, left_bottom, right_bottom, diamond_corners, square_corners
    
def cost_per_node(sum_vertice, cost):
    if sum_vertice == 34:
        cost -= 1
    elif sum_vertice > 0 and sum_vertice <= 34:
        cost -= 0.3
    elif sum_vertice >= 34:
        cost += 10
    return cost
    
def board8(board, fmt=(4 * '{} {} {} {}\n')):
    # print("hello")
    "A string representing an 8-puzzle board"
    print("       ", board[0], "    ")
    print("", board[1], " ", board[2], " ", board[3], " ", board[4], "  ")
    print("", board[5], "          ", board[6], " ")
    print(board[7], "             ", board[8])
    print("", board[9], "          ", board[10], " ")
    print("", board[11], " ", board[12], " ", board[13], " ", board[14], "  ")
    print("       ", board[15], "    ")
    print("\n")

class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height) 
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty
        
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

In [66]:
# Some specific OctaPuzzle problems for aStarSearch

a0 = OctaPuzzle((9,3,13,0,0,0,5,0,14,16,0,0,0,8,0,0))
a1 = OctaPuzzle((9,0,13,0,0,0,5,0,14,16,0,0,0,8,0,0))
a2 = OctaPuzzle((9,0,13,0,0,0,5,0,14,16,0,0,0,0,0,0))
a3 = OctaPuzzle((0,0,13,0,0,0,5,0,14,16,0,0,0,0,0,0))

In the problem (a2) the search finds a different solution than the one given in the pdf, which means our algorithm is definitely working.

```
        9     
 12   13   6   3   
 2            5  
10               14
 16            11  
 4   7   8   15   
        1    
```

In [72]:
for s in path_states(astar_search(a2)):
    board8(s)

        9     
 0   13   0   0   
 0            5  
0               14
 16            0  
 0   0   0   0   
        0     


        9     
 12   13   0   0   
 0            5  
0               14
 16            0  
 0   0   0   0   
        0     


        9     
 12   13   6   0   
 0            5  
0               14
 16            0  
 0   0   0   0   
        0     


        9     
 12   13   6   3   
 0            5  
0               14
 16            0  
 0   0   0   0   
        0     


        9     
 12   13   6   3   
 2            5  
0               14
 16            0  
 0   0   0   0   
        0     


        9     
 12   13   6   3   
 2            5  
10               14
 16            0  
 0   0   0   0   
        0     


        9     
 12   13   6   3   
 2            5  
10               14
 16            11  
 0   0   0   0   
        0     


        9     
 12   13   6   3   
 2            5  
10               14
 16            11  
 4   0   0   0   
     

In [73]:
# Some specific OctaPuzzle problems for breadthFirstSearch

b0 = OctaPuzzle((9,3,13,6,12,0,5,0,14,16,0,0,0,8,15,10))
b1 = OctaPuzzle((9,3,13,0,12,0,5,0,14,16,0,0,0,8,0,10))
b2 = OctaPuzzle((9,3,13,0,12,0,5,0,14,16,0,0,0,8,0,0))
b3 = OctaPuzzle((9,3,13,0,0,0,5,0,14,16,0,0,0,8,0,0))

In [62]:
# Solve an 8 puzzle problem and print out each state

# print(path_states(breadth_first_search(e1)))
for s in path_states(breadth_first_search(b3)):
    # print('hello')
    board8(s)

        9     
 3   13   0   0   
 0            5  
0               14
 16            0  
 0   0   8   0   
        0     


        9     
 3   13   6   0   
 0            5  
0               14
 16            0  
 0   0   8   0   
        0     


        9     
 3   13   6   12   
 0            5  
0               14
 16            0  
 0   0   8   0   
        0     


        9     
 3   13   6   12   
 11            5  
0               14
 16            0  
 0   0   8   0   
        0     


        9     
 3   13   6   12   
 11            5  
1               14
 16            0  
 0   0   8   0   
        0     


        9     
 3   13   6   12   
 11            5  
1               14
 16            2  
 0   0   8   0   
        0     


        9     
 3   13   6   12   
 11            5  
1               14
 16            2  
 4   0   8   0   
        0     


        9     
 3   13   6   12   
 11            5  
1               14
 16            2  
 4   7   8   0   
      

# Reporting Summary Statistics on Search Algorithms
Now let's gather some metrics on how well each algorithm does. We'll use CountCalls to wrap a Problem object in such a way that calls to its methods are delegated to the original problem, but each call increments a counter. Once we've solved the problem, we print out summary statistics.

In [74]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:80])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))

In [69]:
report([astar_search], [a0, a1, a2, a3])

astar_search:
    9,252 nodes |    1,029 goal |    9 cost |   1,037 actions | OctaPuzzle((9, 3, 13, 0, 0, 0, 5, 0, 14, 16, 0, 0, 0, 8, 0, 0), '')
   73,010 nodes |    7,302 goal |   10 cost |   7,311 actions | OctaPuzzle((9, 0, 13, 0, 0, 0, 5, 0, 14, 16, 0, 0, 0, 8, 0, 0), '')
  382,151 nodes |   34,742 goal |   11 cost |  34,752 actions | OctaPuzzle((9, 0, 13, 0, 0, 0, 5, 0, 14, 16, 0, 0, 0, 0, 0, 0), '')
4,717,380 nodes |  393,116 goal |   12 cost | 393,127 actions | OctaPuzzle((0, 0, 13, 0, 0, 0, 5, 0, 14, 16, 0, 0, 0, 0, 0, 0), '')
5,181,793 nodes |  436,189 goal |   42 cost | 436,227 actions | TOTAL



In [75]:
report([breadth_first_search], [b0, b1, b2, b3])

breadth_first_search:
      914 nodes |      915 goal |    5 cost |     188 actions | OctaPuzzle((9, 3, 13, 6, 12, 0, 5, 0, 14, 16, 0, 0, 0, 8, 15, 10), '')
   43,827 nodes |   43,828 goal |    7 cost |   6,268 actions | OctaPuzzle((9, 3, 13, 0, 12, 0, 5, 0, 14, 16, 0, 0, 0, 8, 0, 10), '')
  381,462 nodes |  381,463 goal |    8 cost |  47,691 actions | OctaPuzzle((9, 3, 13, 0, 12, 0, 5, 0, 14, 16, 0, 0, 0, 8, 0, 0), '')
3,739,065 nodes |3,739,066 goal |    9 cost | 415,461 actions | OctaPuzzle((9, 3, 13, 0, 0, 0, 5, 0, 14, 16, 0, 0, 0, 8, 0, 0), '')
4,165,268 nodes |4,165,272 goal |   29 cost | 469,608 actions | TOTAL

